# Timelag Maps
Compute all of the timelag maps here for all heating frequencies and observations and then save the maps.

In [1]:
import os
import sys

import numpy as np
import distributed
import matplotlib.pyplot as plt
import matplotlib.colors
from sunpy.map import Map,GenericMap
from astropy.coordinates import SkyCoord
import astropy.units as u

import synthesizAR
from synthesizAR.instruments import InstrumentSDOAIA

sys.path.append(os.path.join(os.environ['HOME'], 'projects/synthesized_timelag_maps/scripts/'))
from aiacube import DistributedAIACube
from timelags import AIATimeLags

%matplotlib inline

In [2]:
channel_pairs = [(94,335),
                 (94,171),
                 (94,193),
                 (94,131),
                 (94,211),
                 (335,131),
                 (335,193),
                 (335,211), 
                 (335,171),
                 (211,131),
                 (211,171),
                 (211,193),
                 (193,171),
                 (193,131),
                 (171,131),]

In [3]:
timelag_bounds = (-6*u.hour,6*u.hour)

In [4]:
channels = [94,131,171,193,211,335]
read_template = '/storage-home/w/wtb2/data/timelag_synthesis_v2/{}/nei/SDO_AIA/{}/map_t{:06d}.fits'
save_template = '/storage-home/w/wtb2/data/shine2018/{}_{}_{}.fits'

In [5]:
cluster = distributed.LocalCluster(n_workers=64,threads_per_worker=1)
client = distributed.Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:42871 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 64 Cores: 64 Memory: 270.38 GB


## Models

In [6]:
frequencies = ['high_frequency','intermediate_frequency','low_frequency',
               'cooling','cooling_outofphase_long']

In [7]:
for f in frequencies:
    irange = range(500,2500) if f!='cooling' else range(0,1000)
    tl_cubes = AIATimeLags(*[DistributedAIACube.from_files([read_template.format(f,c,i) for i in irange])
                             for c in channels])
    for ca,cb in channel_pairs:
        tmap,cmap = tl_cubes.make_timelag_map(float(ca),float(cb),
                                              timelag_bounds=timelag_bounds,
                                              return_correlation_map=True)
        tmap.save(save_template.format(f,f'{ca}-{cb}','timelag'))
        cmap.save(save_template.format(f,f'{ca}-{cb}','correlation'))

## Data
Need to figure out what to do here since interpolating cube in Dask is challenging...